In [1]:
import pandas as pd
import numpy as np

from datetime import datetime


from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [3]:
boston = datasets.load_boston()

In [4]:
for key in boston.keys():
    print( key )

data
target
feature_names
DESCR
filename


In [5]:
X = pd.DataFrame(data=boston.data, columns=boston.feature_names)
y = pd.Series(boston.target)

In [6]:
X.shape

(506, 13)

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB


In [8]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [9]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
dtype: float64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
result = pd.DataFrame(y_test.reset_index(drop=True), columns=['target'])

In [12]:
#Логистическая регрессия

In [13]:
logreg = LinearRegression()

fit_intercepts = [False, True]
param_regr = dict(fit_intercept=fit_intercepts)

In [14]:
grid_reg = GridSearchCV(logreg, param_regr, cv=5)
grid_reg.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [False, True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
print(grid_reg.best_score_)
print(grid_reg.best_estimator_)

0.7245560841037053
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)


In [16]:
result['LinearRegression'] = pd.Series(grid_reg.best_estimator_.predict(X_test))

In [17]:
#Дерево решений

In [18]:
model_tree = DecisionTreeRegressor()

criterions = ['mse', 'friedman_mse', 'mae'] 
min_samples_leafs = range(1,10)
min_weight_fraction_leafs = [0, 0.01, 0.1, 0.15, 0.25, 0.5] 

param_tree = dict(criterion=criterions,
                  min_samples_leaf=min_samples_leafs,
                  min_weight_fraction_leaf=min_weight_fraction_leafs)

In [19]:
grid_tree = GridSearchCV(model_tree, param_tree, cv=5)
grid_tree.fit(X_train, y_train)

D:\l\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['mse', 'friedman_mse', 'mae'], 'min_samples_leaf': range(1, 10), 'min_weight_fraction_leaf': [0, 0.01, 0.1, 0.15, 0.25, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
result['DecisionTreeRegressor'] = pd.Series(grid_tree.best_estimator_.predict(X_test))

In [29]:
#Метод случайных соседей

In [22]:
model_knn = KNeighborsRegressor()

n_neighbor = range(1, 30)
leaf_size = range(5, 30)
weight = ['uniform', 'distance']

#parameters
param_knn = dict(n_neighbors=n_neighbor, weights=weight, 
                     leaf_size=leaf_size)

In [23]:
grid_knn = GridSearchCV(model_knn, param_knn, cv=5)
grid_knn.fit(X_train, y_train)

D:\l\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': range(1, 30), 'weights': ['uniform', 'distance'], 'leaf_size': range(5, 30)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
print(grid_knn.best_score_)
print(grid_knn.best_estimator_)

0.5175768992713482
KNeighborsRegressor(algorithm='auto', leaf_size=5, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=9, p=2,
          weights='distance')


In [25]:
result['KNeighborsRegressor'] = pd.Series(grid_knn.best_estimator_.predict(X_test))

In [26]:
columns = result.columns

traces=[]
for i in columns:
    traces.append(go.Scatter(
                    x=result.index,
                    y=result[i],
                    name=i,
                    orientation = 'v')
                 )

layout = {'title': 'Models result'}
fig = go.Figure(data=traces, layout=layout)

iplot(fig, show_link=False)